<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/Create_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##DataSet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

#Preprocess

In [ ]:
df_text=pd.read_csv('/content/drive/MyDrive/PaperA3/TFIDF/L2_G02')
df_diagnosis=pd.read_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Final_L1.csv')

In [ ]:
print(df_text.shape)
print(df_diagnosis.shape)

(26341, 5)
(52307, 21)


In [ ]:
df_text.head()

,Unnamed: 0,HADM_ID,TFIDF,list_CUIs,number_CUIs
0,0,157267.0,"c0026727,c0585361,c0055856,c0001314,c0151594,c...","['c0026727', 'c0585361', 'c0055856', 'c0001314...",57
1,1,123103.0,"c0336779,c1272695,c0301362,c0024129,c0201950,c...","['c0336779', 'c1272695', 'c0301362', 'c0024129...",28
2,2,139931.0,"c0226332,c0441713,c0232488,c0226333,c0042512,c...","['c0226332', 'c0441713', 'c0232488', 'c0226333...",42
3,3,198295.0,"c0267853,c3275700,c0239981,c3203358,c0162510,c...","['c0267853', 'c3275700', 'c0239981', 'c3203358...",52
4,4,177962.0,"c0334079,c3272265,c0036310,c0014876,c0392689,c...","['c0334079', 'c3272265', 'c0036310', 'c0014876...",40


In [ ]:
filename="/content/drive/MyDrive/PaperA3/CUI/CUI_Total.csv"
df=pd.read_csv(filename)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
dd=df[['first_cuid', 'canonical_name']]
print(len(dd))
dd.drop_duplicates(keep="last",inplace=True)
print(len(dd))

18075318
25312


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:

dd.head()

,first_cuid,canonical_name
355,C1278302,Hepatitis B screening test
400,C0004285,Audiology
833,C0001916,Albinism
1130,C0013069,Double Outlet Right Ventricle
1691,C1394030,Coronal hypospadias


برای تولید دیکشنری به این فرم حتما باید زیپ کرد

In [ ]:

dictionary = dict(zip(dd.first_cuid, dd.canonical_name))


In [ ]:
dictionary

In [ ]:
dd.to_csv('/content/drive/MyDrive/PaperA3/Diagnoses_labels/Dictionary.csv')


In [ ]:
import ast
def complex_function( ini_list):

    res = ast.literal_eval(ini_list)
    tms_text=''
    try:
      for i in res:
        
        if(i.upper().startswith("NOT")):
          tms_text=tms_text +'NOT '+dictionary[i[4:].upper()] +' '
        else:
          tms_text=tms_text +dictionary[i.upper()] +' '  
    except :
          print(i)    
    return tms_text 

In [ ]:
dictionary['C0026727']

'Mucous body substance'

In [ ]:
df_text['Text'] = df_text.apply(lambda x: complex_function(x['list_CUIs']), axis=1)
df_text.head()


,Unnamed: 0,HADM_ID,TFIDF,list_CUIs,number_CUIs,Text
0,0,157267.0,"c0026727,c0585361,c0055856,c0001314,c0151594,c...","['c0026727', 'c0585361', 'c0055856', 'c0001314...",57,Mucous body substance Twice a day clarithromyc...
1,1,123103.0,"c0336779,c1272695,c0301362,c0024129,c0201950,c...","['c0336779', 'c1272695', 'c0301362', 'c0024129...",28,Machine Done (qualifier value) Bromdimethoxyam...
2,2,139931.0,"c0226332,c0441713,c0232488,c0226333,c0042512,c...","['c0226332', 'c0441713', 'c0232488', 'c0226333...",42,Structure of right renal artery Chemical injur...
3,3,198295.0,"c0267853,c3275700,c0239981,c3203358,c0162510,c...","['c0267853', 'c3275700', 'c0239981', 'c3203358...",52,Calculus of gallbladder with cholecystitis Hep...
4,4,177962.0,"c0334079,c3272265,c0036310,c0014876,c0392689,c...","['c0334079', 'c3272265', 'c0036310', 'c0014876...",40,Aplastic Three Vessel Coronary Disease Scheuer...


In [ ]:
df_total = pd.merge(df_text, df_diagnosis, on=["HADM_ID"],how="left")
print(df_total.shape)
df_total.head()

(26341, 26)


,Unnamed: 0_x,HADM_ID,TFIDF,list_CUIs,number_CUIs,Text,Unnamed: 0_y,L1,L_10.0,L_8.0,...,L_1.0,L_14.0,L_3.0,L_9.0,L_17.0,L_15.0,L_13.0,L_16.0,L_7.0,L_12.0
0,0,157267.0,"c0026727,c0585361,c0055856,c0001314,c0151594,c...","['c0026727', 'c0585361', 'c0055856', 'c0001314...",57,Mucous body substance Twice a day clarithromyc...,29918.0,13.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,123103.0,"c0336779,c1272695,c0301362,c0024129,c0201950,c...","['c0336779', 'c1272695', 'c0301362', 'c0024129...",28,Machine Done (qualifier value) Bromdimethoxyam...,12144.0,18.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,139931.0,"c0226332,c0441713,c0232488,c0226333,c0042512,c...","['c0226332', 'c0441713', 'c0232488', 'c0226333...",42,Structure of right renal artery Chemical injur...,20902.0,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,198295.0,"c0267853,c3275700,c0239981,c3203358,c0162510,c...","['c0267853', 'c3275700', 'c0239981', 'c3203358...",52,Calculus of gallbladder with cholecystitis Hep...,51440.0,25.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,4,177962.0,"c0334079,c3272265,c0036310,c0014876,c0392689,c...","['c0334079', 'c3272265', 'c0036310', 'c0014876...",40,Aplastic Three Vessel Coronary Disease Scheuer...,40804.0,21.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_total.columns


Index(['Unnamed: 0_x', 'HADM_ID', 'TFIDF', 'list_CUIs', 'number_CUIs', 'Text',
       'Unnamed: 0_y', 'L1', 'L_10.0', 'L_8.0', 'L_6.0', 'L_4.0', 'L_11.0',
       'L_5.0', 'L_2.0', 'L_18.0', 'L_1.0', 'L_14.0', 'L_3.0', 'L_9.0',
       'L_17.0', 'L_15.0', 'L_13.0', 'L_16.0', 'L_7.0', 'L_12.0'],
      dtype='object')

In [ ]:
df_total.drop(['L1','Unnamed: 0_x',  'TFIDF', 'list_CUIs', 'number_CUIs','Unnamed: 0_y'], axis=1, inplace=True)
df_total.columns



Index(['HADM_ID', 'Text', 'L_10.0', 'L_8.0', 'L_6.0', 'L_4.0', 'L_11.0',
       'L_5.0', 'L_2.0', 'L_18.0', 'L_1.0', 'L_14.0', 'L_3.0', 'L_9.0',
       'L_17.0', 'L_15.0', 'L_13.0', 'L_16.0', 'L_7.0', 'L_12.0'],
      dtype='object')

In [ ]:

df_raw=df_total
df_raw = df_raw.rename(columns={'Text': 'comment_text'})

In [ ]:
df_raw.shape

(26341, 20)

In [55]:
def bool(value):    
  if(value==1):
          tms_text=True
  else:
          tms_text=False    
  return tms_text 




In [56]:
df_raw['L_1.0'] = df_raw.apply(lambda x: bool(x['L_1.0']), axis=1)  
df_raw['L_2.0'] = df_raw.apply(lambda x: bool(x['L_2.0']), axis=1)  
df_raw['L_3.0'] = df_raw.apply(lambda x: bool(x['L_3.0']), axis=1)  
df_raw['L_4.0'] = df_raw.apply(lambda x: bool(x['L_4.0']), axis=1) 
df_raw['L_5.0'] = df_raw.apply(lambda x: bool(x['L_5.0']), axis=1)  
df_raw['L_6.0'] = df_raw.apply(lambda x: bool(x['L_6.0']), axis=1)  
df_raw['L_7.0'] = df_raw.apply(lambda x: bool(x['L_7.0']), axis=1)  
df_raw['L_8.0'] = df_raw.apply(lambda x: bool(x['L_8.0']), axis=1)  
df_raw['L_9.0'] = df_raw.apply(lambda x: bool(x['L_9.0']), axis=1) 
df_raw['L_10.0'] = df_raw.apply(lambda x: bool(x['L_10.0']), axis=1)  
df_raw['L_11.0'] = df_raw.apply(lambda x: bool(x['L_11.0']), axis=1)  
df_raw['L_12.0'] = df_raw.apply(lambda x: bool(x['L_12.0']), axis=1)  
df_raw['L_13.0'] = df_raw.apply(lambda x: bool(x['L_13.0']), axis=1)  
df_raw['L_14.0'] = df_raw.apply(lambda x: bool(x['L_14.0']), axis=1) 
df_raw['L_15.0'] = df_raw.apply(lambda x: bool(x['L_15.0']), axis=1)  
df_raw['L_16.0'] = df_raw.apply(lambda x: bool(x['L_16.0']), axis=1)  
df_raw['L_17.0'] = df_raw.apply(lambda x: bool(x['L_17.0']), axis=1)  
df_raw['L_18.0'] = df_raw.apply(lambda x: bool(x['L_18.0']), axis=1)  


In [57]:
df_raw.head()

,HADM_ID,comment_text,L_10.0,L_8.0,L_6.0,L_4.0,L_11.0,L_5.0,L_2.0,L_18.0,L_1.0,L_14.0,L_3.0,L_9.0,L_17.0,L_15.0,L_13.0,L_16.0,L_7.0,L_12.0
0,157267.0,Mucous body substance Twice a day clarithromyc...,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,123103.0,Machine Done (qualifier value) Bromdimethoxyam...,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,139931.0,Structure of right renal artery Chemical injur...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,198295.0,Calculus of gallbladder with cholecystitis Hep...,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
4,177962.0,Aplastic Three Vessel Coronary Disease Scheuer...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [58]:
df_raw.to_csv('/content/drive/MyDrive/PaperA3/Final_Dataset/df_raw_TFIDF_L2_G02.csv')